Using a dataset of mfccs with a CNN model.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
import warnings
warnings.simplefilter("ignore")
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from contextlib import redirect_stdout
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve

In [56]:
folder = '/content/drive/My Drive/Colab Notebooks/COVID-19'
if not os.path.exists(folder):
  print(folder + ' does not exist')

mfcc_folder = os.path.join(folder, 'mfcc_json_files')
if not os.path.exists(mfcc_folder):
  print(mfcc_folder + ' does not exist')

Read in json files created previously in Extract_rawMFCCs.ipynb.


In [57]:
def load_jsonfile(path, filename):
  with open(os.path.join(path, filename)) as json_file:
    data = json.load(json_file)
  print("{} loaded".format(filename))
  return data

In [58]:
test_data = load_jsonfile(mfcc_folder, 'test_mfccs.json')
train_data = load_jsonfile(mfcc_folder, 'train_mfccs.json')

test_mfccs.json loaded
train_mfccs.json loaded


Change labels to 0s and 1s

In [59]:
test_data_labels = []
for i in range (len(test_data["labels"])):
  if test_data["labels"][i] == 'healthy':
    test_data_labels.append(1)
  else:
    test_data_labels.append(0)

train_data_labels = []
for i in range (len(train_data["labels"])):
  if train_data["labels"][i] == 'healthy':
    train_data_labels.append(1)
  else:
    train_data_labels.append(0)


Create X_train, y_train, X_test, y_test

In [60]:
# Test data
X_test = np.array(test_data["mfcc"])
y_test = np.array(test_data_labels)

# Train data
X_train = np.array(train_data["mfcc"])
y_train = np.array(train_data_labels)

print("X_train shape: {}, y_train shape: {}, X_test shape: {}, y_test shape: {}".format(
    X_train.shape, y_train.shape, X_test.shape, y_test.shape))


X_train shape: (11401, 301, 13), y_train shape: (11401,), X_test shape: (1000, 301, 13), y_test shape: (1000,)


Split out a validation set

In [61]:
validation_size = 0.2
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_size)

Check shape of X_train, y_train, X_val, y_val, X_test and y_test

In [62]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(9120, 301, 13)
(9120,)
(2281, 301, 13)
(2281,)
(1000, 301, 13)
(1000,)


In [63]:
# Change to 3D ready to feed into CNN
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [64]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(9120, 301, 13, 1)
(9120,)
(2281, 301, 13, 1)
(2281,)
(1000, 301, 13, 1)
(1000,)


Create datasets

In [65]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [66]:
# Make sure the training data is shuffled when fed into the model

BATCH_SIZE = 100
SHUFFLE_BUFFER_SIZE = 10000 # Needs to be bigger than number of samples in trining set

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

# CNN Network



Although there are built in measures for precision and recall, there are not the equivalent measures of specificity and negative prediction rate so I have had to create custom metrics for those.

I have used example in (tf.keras.metrics.Metric | TensorFlow Core v2.7.0) for Precision
 and changed it to calculate the metrics I need.

In [67]:
# Always run this code.
from keras.utils import metrics_utils
from keras.utils.generic_utils import to_list
from keras import backend
class Specificity(tf.keras.metrics.Metric):
  def __init__(self,
               thresholds=None,
               top_k=None,
               class_id=None,
               name=None,
               dtype=None):
    super(Specificity, self).__init__(name=name, dtype=dtype)
    self.init_thresholds = thresholds
    self.top_k = top_k
    self.class_id = class_id

    default_threshold = 0.5 if top_k is None else metrics_utils.NEG_INF
    self.thresholds = metrics_utils.parse_init_thresholds(
        thresholds, default_threshold=default_threshold)
    self._thresholds_distributed_evenly = (
        metrics_utils.is_evenly_distributed_thresholds(self.thresholds))
    self.true_negatives = self.add_weight(
        'true_negatives',
        shape=(len(self.thresholds),),
        initializer=tf.compat.v1.zeros_initializer)
    self.false_positives = self.add_weight(
        'false_positives',
        shape=(len(self.thresholds),),
        initializer=tf.compat.v1.zeros_initializer)

  def update_state(self, y_true, y_pred, sample_weight=None):
    """Accumulates true positive and false positive statistics.
    Args:
      y_true: The ground truth values, with the same dimensions as `y_pred`.
        Will be cast to `bool`.
      y_pred: The predicted values. Each element must be in the range `[0, 1]`.
      sample_weight: Optional weighting of each example. Defaults to 1. Can be a
        `Tensor` whose rank is either 0, or the same rank as `y_true`, and must
        be broadcastable to `y_true`.
    Returns:
      Update op.
    """
    return metrics_utils.update_confusion_matrix_variables(
        {
            metrics_utils.ConfusionMatrix.TRUE_NEGATIVES: self.true_negatives,
            metrics_utils.ConfusionMatrix.FALSE_POSITIVES: self.false_positives
        },
        y_true,
        y_pred,
        thresholds=self.thresholds,
        thresholds_distributed_evenly=self._thresholds_distributed_evenly,
        top_k=self.top_k,
        class_id=self.class_id,
        sample_weight=sample_weight)

  def result(self):
    result = tf.math.divide_no_nan(
        self.true_negatives,
        tf.math.add(self.true_negatives, self.false_positives))
    return result[0] if len(self.thresholds) == 1 else result

  def reset_state(self):
    num_thresholds = len(to_list(self.thresholds))
    backend.batch_set_value([(v, np.zeros((num_thresholds,)))
                             for v in (self.true_negatives,
                                       self.false_positives)])

  def get_config(self):
    config = {
        'thresholds': self.init_thresholds,
        'top_k': self.top_k,
        'class_id': self.class_id
    }
    base_config = super(Specificity, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

In [68]:
# Always run this code.
from keras.utils import metrics_utils
from keras.utils.generic_utils import to_list
from keras import backend
class NPV(tf.keras.metrics.Metric):
  def __init__(self,
               thresholds=None,
               top_k=None,
               class_id=None,
               name=None,
               dtype=None):
    super(NPV, self).__init__(name=name, dtype=dtype)
    self.init_thresholds = thresholds
    self.top_k = top_k
    self.class_id = class_id

    default_threshold = 0.5 if top_k is None else metrics_utils.NEG_INF
    self.thresholds = metrics_utils.parse_init_thresholds(
        thresholds, default_threshold=default_threshold)
    self._thresholds_distributed_evenly = (
        metrics_utils.is_evenly_distributed_thresholds(self.thresholds))
    self.true_negatives = self.add_weight(
        'true_negatives',
        shape=(len(self.thresholds),),
        initializer=tf.compat.v1.zeros_initializer)
    self.false_negatives = self.add_weight(
        'false_negatives',
        shape=(len(self.thresholds),),
        initializer=tf.compat.v1.zeros_initializer)

  def update_state(self, y_true, y_pred, sample_weight=None):
    """Accumulates true positive and false positive statistics.
    Args:
      y_true: The ground truth values, with the same dimensions as `y_pred`.
        Will be cast to `bool`.
      y_pred: The predicted values. Each element must be in the range `[0, 1]`.
      sample_weight: Optional weighting of each example. Defaults to 1. Can be a
        `Tensor` whose rank is either 0, or the same rank as `y_true`, and must
        be broadcastable to `y_true`.
    Returns:
      Update op.
    """
    return metrics_utils.update_confusion_matrix_variables(
        {
            metrics_utils.ConfusionMatrix.TRUE_NEGATIVES: self.true_negatives,
            metrics_utils.ConfusionMatrix.FALSE_NEGATIVES: self.false_negatives
        },
        y_true,
        y_pred,
        thresholds=self.thresholds,
        thresholds_distributed_evenly=self._thresholds_distributed_evenly,
        top_k=self.top_k,
        class_id=self.class_id,
        sample_weight=sample_weight)

  def result(self):
    result = tf.math.divide_no_nan(
        self.true_negatives,
        tf.math.add(self.true_negatives, self.false_negatives))
    return result[0] if len(self.thresholds) == 1 else result

  def reset_state(self):
    num_thresholds = len(to_list(self.thresholds))
    backend.batch_set_value([(v, np.zeros((num_thresholds,)))
                             for v in (self.true_negatives,
                                       self.false_negatives)])

  def get_config(self):
    config = {
        'thresholds': self.init_thresholds,
        'top_k': self.top_k,
        'class_id': self.class_id
    }
    base_config = super(NPV, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

# **Metrics available for use**

In [69]:
# Always run this cell.
precision = tf.keras.metrics.Precision(name = "precision")
recall = tf.keras.metrics.Recall(name = "recall")
specificity = Specificity(name = "specificity")
TP = tf.keras.metrics.TruePositives(name = "TP") 
FP = tf.keras.metrics.FalsePositives(name = "FP")
FN = tf.keras.metrics.FalseNegatives(name = "FN")
TN = tf.keras.metrics.TrueNegatives(name = "TN")
auc = tf.keras.metrics.AUC(name = "auc")
npv = NPV(name = "NPV")

# **Fit model on the training dataset**

In [70]:
#Run this cell to create model or use next cell to load existing model.
model_name = "Covid_Model_m1.h5" #Change name here for each new model

INPUT_SHAPE = (X_train.shape[1], X_train.shape[2], 1)

model = keras.Sequential([              
   
    # Block One
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same',
                  input_shape = INPUT_SHAPE),
    layers.MaxPool2D((3, 3), strides=(2, 2), padding='same'),
    layers.BatchNormalization(),

    # Block Two
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D((3, 3), strides=(2, 2), padding='same'),
    layers.BatchNormalization(),

    # Block Three
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D((3, 3), strides=(2, 2), padding='same'),
    layers.BatchNormalization(),

    # Block Four
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D((3, 3), strides=(2, 2), padding='same'),
    layers.BatchNormalization(),

    # Block Five
    layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D((3, 3), strides=(2, 2), padding='same'),
    layers.BatchNormalization(),

    # Head
    layers.Flatten(),
    layers.Dropout(0.25),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(1, activation='sigmoid'),
])

model._name= model_name[:-3]

model_plots_folder = os.path.join(folder,'model_plots')

# Save model summaries to a text file.
with open(os.path.join(model_plots_folder,'Model_summaries.txt'), 'a') as f:
    with redirect_stdout(f):
        model.summary()

# Save model plots to individual png files.
plot_model(
    model,
    to_file=(os.path.join(model_plots_folder, model_name[:-3] + ".png")),
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
)

model.summary()
plot_model(model, show_shapes=True)

es_callback = EarlyStopping(monitor = 'val_auc',
                            mode='max',
                            min_delta = 0.01,
                            patience = 10,
                            restore_best_weights = True)

model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=0.01),
    loss='binary_crossentropy',
    metrics=[TP, FP, FN, TN, 
             npv, specificity, 
             recall, auc
             ]
)

model_folder = '/content/drive/My Drive/Colab Notebooks/models/'

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    callbacks = [es_callback],
    epochs = 30,
    #steps_per_epoch=len(train_dataset) // BATCH_SIZE,
    #validation_steps=len(val_dataset) // BATCH_SIZE
    )

model.save(model_folder + model_name)


Model: "Covid_Model_m1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_55 (Conv2D)          (None, 301, 13, 32)       320       
                                                                 
 max_pooling2d_55 (MaxPoolin  (None, 151, 7, 32)       0         
 g2D)                                                            
                                                                 
 batch_normalization_55 (Bat  (None, 151, 7, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_56 (Conv2D)          (None, 151, 7, 32)        9248      
                                                                 
 max_pooling2d_56 (MaxPoolin  (None, 76, 4, 32)        0         
 g2D)                                                            
                                                    

In [ ]:
#Only run this cell if you want to reload model run previously.
model_folder = '/content/drive/My Drive/Colab Notebooks/models/'

custom_objects = {"NPV": NPV, "Specificity": Specificity}
with keras.utils.custom_object_scope(custom_objects):
    model = tf.keras.models.load_model(model_folder + "Covid_Model_m1.h5")

# **Apply model to test dataset.**

In [71]:
_,TP,FP,FN,TN, Spec, npv, Recall, AUC = model.evaluate(test_dataset)
print('Confusion Matrix: ', TP, FP, FN, TN)
print('Test NPV: %.4f' % npv)
print('Test Specificity: %.4f' % Spec)
print('Test Sensitivity: %.4f' % Recall)
print('Test ROC-AUC: %.4f' % AUC)

10/10 [==============================] - 0s 8ms/step - loss: 0.3446 - TP: 885.0000 - FP: 113.0000 - FN: 1.0000 - TN: 1.0000 - NPV: 0.5000 - specificity: 0.0088 - recall: 0.9989 - auc: 0.6496
Confusion Matrix:  885.0 113.0 1.0 1.0
Test NPV: 0.0088
Test Specificity: 0.5000
Test Sensitivity: 0.9989
Test ROC-AUC: 0.6496
